## 我想用多进程、多线程实现爬虫

### 普通方法测试一下爬取需要多长时间，然后用多线程、多进程的方法做一个对比

In [1]:
#普通方式爬取网站https://movie.douban.com/top250?start=0
import requests
from bs4 import BeautifulSoup
import time
url= 'https://movie.douban.com/top250?start=0'
header= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
html= requests.get(url,headers= header)
soup= BeautifulSoup(html.text,"lxml")

In [5]:
print(html.text)

<!DOCTYPE html>
<html lang="zh-cmn-Hans" class="ua-windows ua-webkit">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
    <title>
豆瓣电影 Top 250
</title>
    
    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <link rel="apple-touch-icon" href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">
    <link href="https://img3.doubanio.com/f/shire/52c9997d6d42db58eab418e976a14d5f3eff981e/css/douban.css" rel="stylesheet" type="text/css">
    <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" rel="stylesheet" type=

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html class="ua-windows ua-webkit" lang="zh-cmn-Hans">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="webkit" name="renderer"/>
  <meta content="always" name="referrer"/>
  <meta content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" name="google-site-verification"/>
  <title>
   豆瓣电影 Top 250
  </title>
  <meta content="cZdR4xxR7RxmM4zE" name="baidu-site-verification"/>
  <meta content="no-cache" http-equiv="Pragma"/>
  <meta content="Sun, 6 Mar 2005 01:00:00 GMT" http-equiv="Expires"/>
  <link href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link href="https://img3.doubanio.com/f/shire/52c9997d6d42db58eab418e976a14d5f3eff981e/css/douban.css" rel="stylesheet" type="text/css"/>
  <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" rel="stylesheet" type="text/css"/>
  <link 

In [34]:
titles= []
for item in soup.find_all(name="img"):
    titles.append(item.attrs['alt'])
titles.pop()

'扫码下载豆瓣 App'

In [35]:
titles

['肖申克的救赎',
 '霸王别姬',
 '阿甘正传',
 '这个杀手不太冷',
 '美丽人生',
 '泰坦尼克号',
 '千与千寻',
 '辛德勒的名单',
 '盗梦空间',
 '忠犬八公的故事',
 '机器人总动员',
 '三傻大闹宝莱坞',
 '放牛班的春天',
 '海上钢琴师',
 '楚门的世界',
 '星际穿越',
 '大话西游之大圣娶亲',
 '龙猫',
 '熔炉',
 '教父',
 '无间道',
 '疯狂动物城',
 '当幸福来敲门',
 '怦然心动',
 '触不可及']

可以通过上述方法提取出所有电影的名字，接下来要做的事情就是翻页

## 爬虫翻页方法

In [52]:
def get_the_html(url):
    header= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
    html= requests.get(url,headers= header)
    return html.text
def parse(text):
    soup= BeautifulSoup(text, 'lxml')
    #查找电影名称
    titles=[]
    for item in soup.find_all(name="img"):
        titles.append(item.attrs['alt'])
    titles.pop()
    return titles
def main():
    URL= 'https://movie.douban.com/top250?start='
    titles= []
    for i in range(0,226,25):
        url= URL+str(i)
        text= get_the_html(url)
        titles.extend(parse(text)) 
    return titles

In [55]:
#看看不用多线程，多进程，爬取250个电影名需要多长时间
start= time.clock()
titles=main()
end= time.clock()
print("常规方法爬取需要：",end-start)

常规方法爬取需要： 3.398493322029026


## 前期处理阶段用到的代码

In [3]:
# write a demo to show how multiprocessing accelerate programme
import requests
from bs4 import BeautifulSoup
import time
import multiprocessing
from multiprocessing import Pool
import threading
def get_the_html(url):
    header= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
    html= requests.get(url,headers= header)
    return html.text
def parse(text):
    soup= BeautifulSoup(text, 'lxml')
    #查找电影名称
    titles=[]
    for item in soup.find_all(name="img"):
        titles.append(item.attrs['alt'])
    titles.pop()
    return titles
def multipro(offset):
    url= 'https://movie.douban.com/top250?start='+str(offset)
    html= get_the_html(url)
    print(parse(html))
def common():
    URL= 'https://movie.douban.com/top250?start='
    titles= []
    for i in range(0,226,25):
        url= URL+str(i)
        text= get_the_html(url)
        print(parse(text))

### 用多进程

In [ ]:
if __name__=='__main__':
    import time
    multiprocessing.freeze_support()
    start= time.clock()
    p= Pool(processes=4)
    for i in range(0,226,25):
        p.apply_async(multipro,(i,))
    p.close()
    p.join()
    # common()
    end= time.clock() 
    print("耗时：",end-start)

由于jupyter notebook中不知道为什么搞不了多进程，所以在vscode上执行，发现比单线程快了不少

## 用多线程

In [4]:
if __name__=='__main__':
    import time
    start= time.clock()
    threads= []
    for i in range(0,226,25):
        t= threading.Thread(target= multipro, args=(i,)) 
        t.start() 
        threads.append(t)
    for t in threads:
        t.join()
    end= time.clock() 
    print("耗时：",end-start)

['2001太空漫游', '城市之光', '东京物语', '绿里奇迹', '撞车', '梦之安魂曲', '疯狂的麦克斯4：狂暴之路', '聚焦', 'E.T. 外星人', '黑鹰坠落', '这个男人来自地球', '末路狂花', '遗愿清单', '发条橙', '变脸', '国王的演讲', '我爱你', '勇闯夺命岛', '千钧一发', '哈利·波特与密室', '哈利·波特与阿兹卡班的囚徒', '黄金三镖客', '非常嫌疑犯', '驴得水', '秒速5厘米']
['我不是药神', '消失的爱人', '告白', '小森林 夏秋篇', '大鱼', '一一', '请以你的名字呼唤我', '阳光姐妹淘', '爱在黎明破晓前', '射雕英雄传之东成西就', '侧耳倾听', '甜蜜蜜', '红辣椒', '海蒂和爷爷', '驯龙高手', '杀人回忆', '倩女幽魂', '超脱', '7号房的礼物', '恐怖直播', '菊次郎的夏天', '哈利·波特与死亡圣器(下)', '小森林 冬春篇', '爱在日落黄昏时', '风之谷']
['谍影重重2', '牯岭街少年杀人事件', '朗读者', '浪潮', '谍影重重', '疯狂的石头', '战争之王', '地球上的星星', '初恋这件小事', '青蛇', '终结者2：审判日', '奇迹男孩', '爱在午夜降临前', '源代码', '新龙门客栈', '步履不停', '小萝莉的猴神大叔', '无耻混蛋', '釜山行', '再次出发之纽约遇见你', '一次别离', '追随', '新世界', '血钻', '彗星来的那一夜']
['海洋', '模仿游戏', '爆裂鼓手', '雨中曲', '一个叫欧维的男人决定去死', '忠犬八公物语', '燃情岁月', '人工智能', '天书奇谭', '魔女宅急便', '房间', '穿越时空的少女', '黑客帝国3：矩阵革命', '恐怖游轮', '海街日记', '魂断蓝桥', '头号玩家', '猜火车', '完美陌生人', '罗生门', '惊魂记', '功夫', '阿飞正传', '香水', '可可西里']
['被嫌弃的松子的一生', '低俗小说', '沉默的羔羊', '蝴蝶效应', '春光乍泄', '穿条纹睡衣的男孩', '勇敢的心', '天使爱美丽', '剪刀手爱德华', '心灵捕手', '禁闭岛', '布达佩斯大饭

在这个例子中用多线程仿佛更加的快

### 普通方法

In [6]:
if __name__=='__main__':
    import time
    start= time.clock()
    common()
    end= time.clock() 
    print("耗时：",end-start)

['肖申克的救赎', '霸王别姬', '阿甘正传', '这个杀手不太冷', '美丽人生', '泰坦尼克号', '千与千寻', '辛德勒的名单', '盗梦空间', '忠犬八公的故事', '机器人总动员', '三傻大闹宝莱坞', '放牛班的春天', '海上钢琴师', '楚门的世界', '星际穿越', '大话西游之大圣娶亲', '龙猫', '熔炉', '教父', '无间道', '疯狂动物城', '当幸福来敲门', '怦然心动', '触不可及']
['蝙蝠侠：黑暗骑士', '乱世佳人', '活着', '控方证人', '少年派的奇幻漂流', '摔跤吧！爸爸', '指环王3：王者无敌', '天空之城', '鬼子来了', '十二怒汉', '天堂电影院', '飞屋环游记', '大话西游之月光宝盒', '哈尔的移动城堡', '罗马假日', '搏击俱乐部', '末代皇帝', '寻梦环游记', '闻香识女人', '辩护人', '素媛', '窃听风暴', '死亡诗社', '两杆大烟枪', '飞越疯人院']
['指环王2：双塔奇兵', '教父2', '狮子王', '指环王1：魔戒再现', 'V字仇杀队', '美丽心灵', '饮食男女', '何以为家', '海豚湾', '情书', '大闹天宫', '钢琴家', '哈利·波特与魔法石', '本杰明·巴顿奇事', '看不见的客人', '黑客帝国', '西西里的美丽传说', '小鞋子', '让子弹飞', '美国往事', '拯救大兵瑞恩', '音乐之声', '猫鼠游戏', '致命魔术', '七宗罪']
['被嫌弃的松子的一生', '低俗小说', '沉默的羔羊', '蝴蝶效应', '春光乍泄', '穿条纹睡衣的男孩', '勇敢的心', '天使爱美丽', '剪刀手爱德华', '心灵捕手', '禁闭岛', '布达佩斯大饭店', '阿凡达', '入殓师', '幽灵公主', '加勒比海盗', '摩登时代', '致命ID', '断背山', '阳光灿烂的日子', '重庆森林', '第六感', '狩猎', '喜剧之王', '玛丽和马克思']
['我不是药神', '消失的爱人', '告白', '小森林 夏秋篇', '大鱼', '一一', '请以你的名字呼唤我', '阳光姐妹淘', '爱在黎明破晓前', '射雕英雄传之东成西就', '侧耳倾听', '甜蜜蜜', '红辣椒'

看来多线程会乱序啊